In [1]:
import time
import requests
import json
import os
import mysql.connector
from utilities.sha_tools import generate_sha_digest
from utilities import logger
from datetime import datetime, timedelta
#import ConfigReader
from urllib.parse import urlencode
from utilities.sha_tools import generate_sha_digest
from urllib.parse import urlencode

# from sqlalchemy import Column, Integer, String, Text, Date
# from sqlalchemy.ext.declarative import declarative_base
# from sqlalchemy import create_engine
# from sqlalchemy.orm import sessionmaker
import datetime
# import sqlite3
from pathlib import Path
import chardet

In [2]:
api_user = 'h=Czxr?m'
api_password = 'B537AF1C-64F9-FB2D-DC31-1E8C334E8D79'

mylogger = logger.Logger(name='TAPD_ana', debug=True).logger
mysql_db_config = {
    "host": "10.101.9.50",
    "user": 'root',
    "password": 'newpass',
    "database": 'requirements'
}

db_connection = mysql.connector.connect(**mysql_db_config)
db_cursor = db_connection.cursor()
if db_connection.is_connected():
    mylogger.debug("Successfully connect mysql database")

2024-05-09 14:25:00.031 | DEBUG    | __main__:<module>:15 - Successfully connect mysql database


In [5]:
def fetch_categories(workspace_id):      
    try:
        r = requests.get(f"https://api.tapd.cn/tcase_categories/count?workspace_id={workspace_id}", auth=(api_user, api_password))
        ret = r.text # 获取接口返回结果
        print(ret) # Python 3
        json_string = json.loads(ret)
        total_entries = json_string['data']['count']
        mylogger.info(f'project {workspace_id} has {total_entries} tcase_categories')
    except Exception as e:
        mylogger.error(f"Something goes wrong {e}")


    base_url = "https://api.tapd.cn/tcase_categories"
    page_to_break = 0
    current_page = 1
    if total_entries >= 50:
       batch_size = 50
    else:
       batch_size =25
    current_index = 0
    item_number = 0 
    current_batch = []
    while current_index < total_entries:
        mylogger.debug(f">>>>>>>>>>Fetch page #{current_page}")
        params = {
            "workspace_id": workspace_id,
            "page":current_page,
            "limit": batch_size
        }

        query_string = urlencode(params)
        full_url = f"{base_url}?{query_string}"
    
        r = requests.get(full_url, auth=(api_user, api_password))
        # r = requests.get(f"https://api.tapd.cn/tcase_categories?workspace_id={workspace_id}", auth=(api_user, api_password))
        ret = r.text # 获取接口返回结果
        decoded_data = json.loads(ret)
        if(decoded_data['status'] != 1):
            mylogger.error(decoded_data)
            break
        else:
            tcases_entries = decoded_data["data"]
            mylogger.debug(f"number of tcases pages are {len(tcases_entries)}")
        
            tcases_list = [item for item in tcases_entries]
            #mylogger.info(tcases_list)
            for tcase in tcases_list:
                try:
                    category_id = tcase['TcaseCategory']['id']
                
                    workspace_id = tcase['TcaseCategory']['workspace_id']
                    parent_id = tcase['TcaseCategory']['parent_id']
                    description= tcase['TcaseCategory']['description']
                    name = tcase['TcaseCategory']['name']
                
                    current_batch.append({'workspace_id': workspace_id,'category_id': category_id, 'name': name, 'description':description, 'parent_id': parent_id})

                
                except Exception as e:
                    print(f"Error processing tcase_categories: {e}")
        
            current_index += batch_size
            current_page += 1
        
           
            time.sleep(1)
            if current_page % 10 == 0:
                time.sleep(20)
    current_time = datetime.datetime.now()
    

    for index in current_batch:
        print(index)
        operation_time= current_time
        query = '''
            INSERT INTO tcase_categories (workspace_id,category_id,name,parent_id)
            VALUES (%s, %s, %s, %s)
            '''
        db_cursor.execute(query, (index['workspace_id'],index['category_id'],index['name'],index['parent_id']))
    
    db_connection.commit()
    # db_cursor.close()
    # db_connection.close()  


           

In [ ]:
path = Path('workspace.txt')
encoding = chardet.detect(path.read_bytes())['encoding']
workspace_id = path.read_text(encoding = encoding)
Workspace_id = workspace_id.splitlines()   

for workspace in Workspace_id:
    fetch_categories(workspace)

In [7]:
def fetch_testcases(workspace_id):    
    try:
        r = requests.get(f"https://api.tapd.cn/tcases/count?workspace_id={workspace_id}", auth=(api_user, api_password))
        ret = r.text # 获取接口返回结果
        #print(ret) # Python 3
        json_string = json.loads(ret)
        total_entries = json_string['data']['count']
        mylogger.info(f'project {workspace_id} has {total_entries} tcases')
    except Exception as e:
        mylogger.error(f"Something goes wrong {e}")

    base_url = "https://api.tapd.cn/tcases"
    page_to_break = 0
    current_page = 1
    if total_entries >= 1000:
        batch_size = 200
    else:
        batch_size =100
    current_index = 0
    item_number = 0 
    current_batch = []
    while current_index < total_entries:
        mylogger.debug(f">>>>>>>>>>Fetch page #{current_page}")
        params = {
            "workspace_id": workspace_id,
            "page":current_page,
            "limit": batch_size
        }

        query_string = urlencode(params)
        full_url = f"{base_url}?{query_string}"
    
        r = requests.get(full_url, auth=(api_user, api_password))
        # r = requests.get(f"https://api.tapd.cn/tapd_wikis?workspace_id={workspace_id}", auth=(api_user, api_password))
        ret = r.text # 获取接口返回结果
        decoded_data = json.loads(ret)
        if(decoded_data['status'] != 1):
            mylogger.error(decoded_data)
            break
        else:
            tcases_entries = decoded_data["data"]
            mylogger.debug(f"number of tcases pages are {len(tcases_entries)}")
            #mylogger.info(tcases_entries)
            tcases_list = [item for item in tcases_entries]
            for tcase in tcases_list:
                try:
                    id = tcase['Tcase']['id']
                    steps = tcase['Tcase']['steps']
                    workspace_id = tcase['Tcase']['workspace_id']
                    category_id = tcase['Tcase']['category_id']
                    url= f"https://www.tapd.cn/{workspace_id}/sparrow/tcase/view/{id}?"
                    name = tcase['Tcase']['name']
                    precondition = tcase['Tcase']['precondition']
                    expectation = tcase['Tcase']['expectation']
                    type = tcase['Tcase']['type']
                    priority = tcase['Tcase']['priority']
                    current_batch.append({'workspace_id': workspace_id,'tcase_id': id, 'category_id': category_id,'name': name,'steps': steps, 'precondition': precondition, 'expectation': expectation, 'URL': url, 'type': type})
                except Exception as e:
                    print(f"Error processing tcase: {e}")
 
            current_index += batch_size
            current_page += 1
        
           
            time.sleep(1)
            if current_page % 10 == 0:
                time.sleep(20)

    for index in current_batch:  
        query = '''
            INSERT INTO tcases (workspace_id,tcase_id,category_id,name,steps,precondition,expectation,URL,type)
            VALUES (%s, %s, %s, %s, %s,%s, %s, %s, %s)
            '''
        db_cursor.execute(query, (index['workspace_id'],index['tcase_id'],index['category_id'],index['name'],index['steps'],index['precondition'],index['expectation'],index['URL'],index['type']))
    
    db_connection.commit()    


                   
           

In [ ]:
path = Path('workspace.txt')
encoding = chardet.detect(path.read_bytes())['encoding']
workspace_id = path.read_text(encoding = encoding)
Workspace_id = workspace_id.splitlines()   

for idx, workspace in enumerate(Workspace_id):
    mylogger.info(f'Start to process project {workspace} {idx+1}/{len(Workspace_id)}')
    fetch_testcases(workspace)

In [12]:
from bs4 import BeautifulSoup
from tqdm import tqdm

def html_to_text(html):
    if not html or len(html) == 0:
        return ''
    
    #"抛弃HTML标签，只提取所有文本内容"
    soup = BeautifulSoup(html, "html.parser")
    text = soup.get_text()
    return text

In [13]:
query = "SELECT id, name, steps FROM tcases"
db_cursor.execute(query)
rows = db_cursor.fetchall()

for row in tqdm(rows, total=len(rows)):
    # 获取name和steps列的值
    name = row[1]
    steps = row[2]
    
    # 合并name和steps
    content = "用例名称是："+ name + "\n用例步骤如下：" + html_to_text(steps)
    finger = generate_sha_digest(content)
    
    # 更新content列的值
    update_query = "UPDATE tcases SET content=%s, finger_print=%s WHERE id = %s"
    db_cursor.execute(update_query, (content, finger, row[0]))

db_connection.commit()


  3%|▎         | 577/19842 [00:04<02:12, 144.94it/s]C:\Users\NstcUser\AppData\Local\Temp\ipykernel_38828\2449968298.py:9: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(html, "html.parser")
100%|██████████| 19842/19842 [02:08<00:00, 153.85it/s]
